In [91]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [92]:
def simple_updown(dataframe):
    dataframe['UD_Trend'] = dataframe['Change'].map(lambda x: 1 if x > 0 else -1)  # 가격이 전날 대비 상승했으면 1, 같거나 하락했으면 -1 로 레이블링한다.
    dataframe['UD_Trend'] = dataframe['UD_Trend'].shift(-1)  # 다음날 트렌드를 예측해야하므로 다음날 트렌드를 앞으로 한 행 당긴다
    dataframe.dropna(inplace=True)  # 결측치 제거
    return dataframe

In [93]:
dataframe = fdr.DataReader('005930', '2000')     # Index is already 'Date'
dataframe.dropna(inplace=True)
dataframe

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2000-01-04,6000,6110,5660,6110,1483967,0.148496
2000-01-05,5800,6060,5520,5580,1493604,-0.086743
2000-01-06,5750,5780,5580,5620,1087810,0.007168
2000-01-07,5560,5670,5360,5540,806195,-0.014235
2000-01-10,5600,5770,5580,5770,937615,0.041516
...,...,...,...,...,...,...
2021-10-20,70700,71000,70200,70300,10151638,-0.004249
2021-10-21,70300,70600,70000,70200,10891094,-0.001422
2021-10-22,70000,70700,70000,70400,8395448,0.002849


In [94]:
dataframe = simple_updown(dataframe)
up_count = dataframe[dataframe['UD_Trend']==1].UD_Trend.count()
total_count = dataframe.UD_Trend.count()
bollinger['UD_Trend'].value_counts()

-1.0    2727
 1.0    2637
Name: UD_Trend, dtype: int64

In [103]:
target_df = dataframe.reset_index()
target_df.drop(['Date'], axis=1, inplace=True)
target_df

,Close,Change,UD_Trend
0,6110,0.148496,-1.0
1,5580,-0.086743,1.0
2,5620,0.007168,-1.0
3,5540,-0.014235,1.0
4,5770,0.041516,-1.0
...,...,...,...
5379,70600,0.005698,-1.0
5380,70300,-0.004249,-1.0
5381,70200,-0.001422,1.0
5382,70400,0.002849,-1.0


In [107]:
train_set, test_set = train_test_split(target_df, test_size=0.2, random_state=42)

X_train = np.array(train_set.loc[:, train_set.columns])
y_train = np.array(train_set.loc[:, ['UD_Trend']])

X_test = np.array(test_set.loc[:, test_set.columns])
y_test = np.array(test_set.loc[:, ['UD_Trend']])

In [108]:
svm_cla = SVC()
svm_cla.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC()

In [109]:
print("train_set score: ", svm_cla.score(X_train, y_train))
print("test_set score  : ", svm_cla.score(X_test, y_test))

train_set score:  0.5177617831437196
test_set score  :  0.4976787372330548
